Initialize the script by importing the right packages and enabling proper settings

In [2]:
# Mute warnings for torchvision v2
import torchvision
torchvision.disable_beta_transforms_warning()

# Importing necessary packags
import numpy as np
import torch
from torchvision.models import googlenet
import torchvision.transforms.v2 as F  
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

from tools.dataset import FacialExpressionDataset
from tools.engine import train_epoch, val_epoch

# Enabling autoreload
%load_ext autoreload
%autoreload 2

# Define device as GPU if cuda available, otherwise cpu
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

NameError: name 'torchvision' is not defined

Define datasets for training and validation

In [ ]:
dataset_train = FacialExpressionDataset(train_transform=None)
dataset_val = FacialExpressionDataset()

if not "indices" in locals():
    indices = torch.randperm(len(dataset_train)).tolist()

dataset_train = torch.utils.data.Subset(dataset_train, indices[:(round(len(indices)*0.75))])
dataset_val = torch.utils.data.Subset(dataset_val, indices[(round(len(indices)*0.75)):])

Define dataloaders 

In [ ]:
BATCH_SIZE = 32
train_loader = DataLoader(dataset_train, BATCH_SIZE, 
                          num_workers=0, shuffle=True,
                            pin_memory=True)

test_laoder = DataLoader(dataset_train, BATCH_SIZE, 
                          num_workers=0, shuffle=False,
                            pin_memory=True)

Create the model

In [ ]:
# Import googlenet with pretrained weights from torchvision
model = googlenet(weights="DEFAULT")

# Modify the last layer to output the amount of classes we have
model.fc = torch.nn.Linear(in_features=model.fc.in_features,
                           out_features=len(dataset_train.dataset.classes))
model = model.to(device)

Define the optimizer and loss function

In [ ]:
# Adam with default parameters
optimizer = Adam(model.parameters())

# Cross-entropy loss for multiclass classification
loss_fn = CrossEntropyLoss().to(device)

Train for 100 epochs

In [ ]:
for epoch in range(0, 100):

    train_loss, train_acc = train_epoch(model, train_loader, device, optimizer, loss_fn)
    val_loss, val_acc = val_epoch(model, train_loader, device, optimizer, loss_fn)

    print(f"Epoch {epoch:4d}  train_loss {train_loss:.4f}  train_acc {train_acc:.2f}%  train_loss {val_loss:.4f}  train_acc {val_acc:.2f}%")

c:\Users\toivo\anaconda3\Lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\toivo\anaconda3\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warni

TypeError: pic should be PIL Image. Got <class 'torch.Tensor'>